In [ ]:
!pip install -q opencv-python

In [ ]:
import shutil
shutil.rmtree("/content/dataset")

In [ ]:
!pip install kaggle


In [ ]:
!kaggle datasets download -d hasyimabdillah/workoutfitness-video

Dataset URL: https://www.kaggle.com/datasets/hasyimabdillah/workoutfitness-video
License(s): CC-BY-NC-SA-4.0
100% 4.31G/4.32G [01:04<00:00, 121MB/s] 
100% 4.32G/4.32G [01:04<00:00, 72.0MB/s]


In [ ]:
import zipfile
import os

# Path to the ZIP archive
zip_file_path = "/content/workoutfitness-video.zip"

# List of folders to extract
folders_to_extract = ['pull Up/']

# Directory to extract the folders into
extraction_directory = "/content/"

# Extract specified folders
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    for folder in folders_to_extract:
        zip_ref.extractall(extraction_directory, members=[f for f in zip_ref.filelist if f.filename.startswith(folder)])


In [ ]:
import os

# Specify the path of the folder containing the files
folder_path = '/content/pull Up'

# Iterate through the range of indices from 1 to 10
for i in range(7, 27):
    # Construct the file path for each file
    file_path = os.path.join(folder_path, f"pull up_{i}.mp4")

    # Check if the file exists before attempting to remove it
    if os.path.exists(file_path):
        # Remove the file
        os.remove(file_path)
    else:
        print(f"File '{file_path}' does not exist.")

In [ ]:
import csv
import cv2
import itertools
import numpy as np
import pandas as pd
import os
import sys
import tempfile
import tqdm

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# import os
# import cv2

# def video_to_images(video_path, output_folder, frame_rate=2):
#     """
#     Convert a video file into a sequence of images.
#     Args:
#     - video_path (str): Path to the input video file.
#     - output_folder (str): Path to the output folder where images will be saved.
#     - frame_rate (int): Rate of frames to be extracted per second (default is 2).
#     Returns:
#     - int: Number of images extracted.
#     """
#     # Create output folder if it doesn't exist
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)

#     # Open the video file
#     video_capture = cv2.VideoCapture(video_path)

#     # Get total number of frames in the video
#     total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

#     # Set the frame rate
#     frame_rate_to_use = frame_rate  # The desired frame rate to use
#     frame_interval = int(video_capture.get(cv2.CAP_PROP_FPS)) // frame_rate_to_use

#     # Read and save frames
#     current_frame = 0
#     images_extracted = 0  # Counter for the number of images extracted
#     while True:
#         # Read the next frame
#         ret, frame = video_capture.read()

#         if not ret:
#             break

#         # Save frame only if it satisfies the frame_interval condition
#         if current_frame % frame_interval == 0:
#             image_filename = os.path.splitext(os.path.basename(video_path))[0] + f"_frame_{current_frame}.jpg"
#             image_path = os.path.join(output_folder, image_filename)
#             cv2.imwrite(image_path, frame)
#             images_extracted += 1

#         current_frame += 1

#     # Release the video capture object
#     video_capture.release()

#     return images_extracted  # Return the count of images extracted

# # Function to process multiple videos in a folder
# def process_videos_in_folder(folder_path, output_folder, frame_rate=2):
#     """
#     Process all videos in a folder and convert them into sequences of images.
#     Args:
#     - folder_path (str): Path to the folder containing input video files.
#     - output_folder (str): Path to the output folder where images will be saved.
#     - frame_rate (int): Rate of frames to be extracted per second (default is 2).
#     Returns:
#     - None
#     """
#     # Create output folder if it doesn't exist
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)

#     total_images_extracted = 0  # Counter for the total number of images extracted

#     # Iterate over all files in the input folder
#     for filename in os.listdir(folder_path):
#         if filename.endswith((".avi", ".mp4", ".MOV")):  # Add more video formats if needed
#             video_path = os.path.join(folder_path, filename)
#             images_extracted = video_to_images(video_path, output_folder, frame_rate)
#             total_images_extracted += images_extracted

#     print("Total images extracted:", total_images_extracted)

# # Example usage:
# videos_folder = "/content/push-up"  # Replace with the path to your videos folder
# output_folder = "/content/data/False"  # Output folder for images
# frame_rate = 5  # Extract one frame per second

# # Process all videos in the folder
# process_videos_in_folder(videos_folder, output_folder, frame_rate)





Total images extracted: 150


In [ ]:
import os
import cv2

def video_to_images(video_path, output_folder, frame_rate=2):
    """
    Convert a video file into a sequence of images.
    Args:
    - video_path (str): Path to the input video file.
    - output_folder (str): Path to the output folder where images will be saved.
    - frame_rate (int): Rate of frames to be extracted per second (default is 2).
    Returns:
    - int: Number of images extracted.
    """
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get video name without extension
    video_name = os.path.splitext(os.path.basename(video_path))[0]

    # Open the video file
    video_capture = cv2.VideoCapture(video_path)

    # Get total number of frames in the video
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

    # Set the frame rate
    frame_rate_to_use = frame_rate  # The desired frame rate to use
    frame_interval = int(video_capture.get(cv2.CAP_PROP_FPS)) // frame_rate_to_use

    # Read and save frames
    current_frame = 0
    images_extracted = 0  # Counter for the number of images extracted
    while True:
        # Read the next frame
        ret, frame = video_capture.read()

        if not ret:
            break

        # Save frame only if it satisfies the frame_interval condition
        if current_frame % frame_interval == 0:
            frame_filename = f"{video_name}_frame_{current_frame:04d}.jpg"  # Padding frame number with zeros
            image_path = os.path.join(output_folder, frame_filename)
            cv2.imwrite(image_path, frame)
            images_extracted += 1

        current_frame += 1

    # Release the video capture object
    video_capture.release()

    return images_extracted  # Return the count of images extracted

# Function to process multiple videos in a folder
def process_videos_in_folder(folder_path, output_folder, frame_rate=2):
    """
    Process all videos in a folder and convert them into sequences of images.
    Args:
    - folder_path (str): Path to the folder containing input video files.
    - output_folder (str): Path to the output folder where images will be saved.
    - frame_rate (int): Rate of frames to be extracted per second (default is 2).
    Returns:
    - None
    """
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    total_images_extracted = 0  # Counter for the total number of images extracted

    # Iterate over all files in the input folder
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith((".avi", ".mp4", ".MOV")):  # Add more video formats if needed
            video_path = os.path.join(folder_path, filename)
            images_extracted = video_to_images(video_path, output_folder, frame_rate)
            total_images_extracted += images_extracted

    print("Total images extracted:", total_images_extracted)

# Example usage:
videos_folder =  "/content/False"  # Replace with the path to your videos folder
output_folder = "/content/dataset/Incorrect"  # Output folder for images
frame_rate = 5  # Extract one frame per second

# Process all videos in the folder
process_videos_in_folder(videos_folder, output_folder, frame_rate)



Total images extracted: 88


In [ ]:
# Download model from TF Hub and check out inference code from GitHub
!wget -q -O movenet_thunder.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite
!git clone https://github.com/tensorflow/examples.git
pose_sample_rpi_path = os.path.join(os.getcwd(), 'examples/lite/examples/pose_estimation/raspberry_pi')
sys.path.append(pose_sample_rpi_path)

# Load MoveNet Thunder model
import utils
from data import BodyPart
from ml import Movenet
movenet = Movenet('movenet_thunder')

# Define function to run pose estimation using MoveNet Thunder.
# You'll apply MoveNet's cropping algorithm and run inference multiple times on
# the input image to improve pose estimation accuracy.
def detect(input_tensor, inference_count=3):
  """Runs detection on an input image.

  Args:
    input_tensor: A [height, width, 3] Tensor of type tf.float32.
      Note that height and width can be anything since the image will be
      immediately resized according to the needs of the model within this
      function.
    inference_count: Number of times the model should run repeatly on the
      same input image to improve detection accuracy.

  Returns:
    A Person entity detected by the MoveNet.SinglePose.
  """
  image_height, image_width, channel = input_tensor.shape

  # Detect pose using the full input image
  movenet.detect(input_tensor.numpy(), reset_crop_region=True)

  # Repeatedly using previous detection result to identify the region of
  # interest and only croping that region to improve detection accuracy
  for _ in range(inference_count - 1):
    person = movenet.detect(input_tensor.numpy(),
                            reset_crop_region=False)

  return person

Cloning into 'examples'...
remote: Enumerating objects: 23745, done.
remote: Total 23745 (delta 0), reused 0 (delta 0), pack-reused 23745
Receiving objects: 100% (23745/23745), 44.08 MiB | 28.82 MiB/s, done.
Resolving deltas: 100% (12948/12948), done.


In [ ]:
def draw_prediction_on_image(
    image, person, crop_region=None, close_figure=True,
    keep_input_size=False):
  """Draws the keypoint predictions on image.

  Args:
    image: An numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    person: A person entity returned from the MoveNet.SinglePose model.
    close_figure: Whether to close the plt figure after the function returns.
    keep_input_size: Whether to keep the size of the input image.

  Returns:
    An numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  # Draw the detection result on top of the image.
  image_np = utils.visualize(image, [person])

  # Plot the image with detection results.
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  im = ax.imshow(image_np)

  if close_figure:
    plt.close(fig)

  if not keep_input_size:
    image_np = utils.keep_aspect_ratio_resizer(image_np, (512, 512))

  return image_np

In [ ]:
class MoveNetPreprocessor(object):
  """Helper class to preprocess pose sample images for classification."""

  def __init__(self,
               images_in_folder,
               images_out_folder,
               csvs_out_path):
    """Creates a preprocessor to detection pose from images and save as CSV.

    Args:
      images_in_folder: Path to the folder with the input images. It should
        follow this structure:
        yoga_poses
        |__ downdog
            |______ 00000128.jpg
            |______ 00000181.bmp
            |______ ...
        |__ goddess
            |______ 00000243.jpg
            |______ 00000306.jpg
            |______ ...
        ...
      images_out_folder: Path to write the images overlay with detected
        landmarks. These images are useful when you need to debug accuracy
        issues.
      csvs_out_path: Path to write the CSV containing the detected landmark
        coordinates and label of each image that can be used to train a pose
        classification model.
    """
    self._images_in_folder = images_in_folder
    self._images_out_folder = images_out_folder
    self._csvs_out_path = csvs_out_path
    self._messages = []

    # Create a temp dir to store the pose CSVs per class
    self._csvs_out_folder_per_class = tempfile.mkdtemp()

    # Get list of pose classes and print image statistics
    self._pose_class_names = sorted(
        [n for n in os.listdir(self._images_in_folder) if not n.startswith('.')]
        )

  def process(self, per_pose_class_limit=None, detection_threshold=0.1):
    """Preprocesses images in the given folder.
    Args:
      per_pose_class_limit: Number of images to load. As preprocessing usually
        takes time, this parameter can be specified to make the reduce of the
        dataset for testing.
      detection_threshold: Only keep images with all landmark confidence score
        above this threshold.
    """
    # Loop through the classes and preprocess its images
    for pose_class_name in self._pose_class_names:
      print('Preprocessing', pose_class_name, file=sys.stderr)

      # Paths for the pose class.
      images_in_folder = os.path.join(self._images_in_folder, pose_class_name)
      images_out_folder = os.path.join(self._images_out_folder, pose_class_name)
      csv_out_path = os.path.join(self._csvs_out_folder_per_class,
                                  pose_class_name + '.csv')
      if not os.path.exists(images_out_folder):
        os.makedirs(images_out_folder)

      # Detect landmarks in each image and write it to a CSV file
      with open(csv_out_path, 'w') as csv_out_file:
        csv_out_writer = csv.writer(csv_out_file,
                                    delimiter=',',
                                    quoting=csv.QUOTE_MINIMAL)
        # Get list of images
        image_names = sorted(
            [n for n in os.listdir(images_in_folder) if not n.startswith('.')])
        if per_pose_class_limit is not None:
          image_names = image_names[:per_pose_class_limit]

        valid_image_count = 0

        # Detect pose landmarks from each image
        for image_name in tqdm.tqdm(image_names):
          image_path = os.path.join(images_in_folder, image_name)

          try:
            image = tf.io.read_file(image_path)
            image = tf.io.decode_jpeg(image)
          except:
            self._messages.append('Skipped ' + image_path + '. Invalid image.')
            continue
          else:
            image = tf.io.read_file(image_path)
            image = tf.io.decode_jpeg(image)
            image_height, image_width, channel = image.shape

          # Skip images that isn't RGB because Movenet requires RGB images
          if channel != 3:
            self._messages.append('Skipped ' + image_path +
                                  '. Image isn\'t in RGB format.')
            continue
          person = detect(image)

          # Save landmarks if all landmarks were detected
          min_landmark_score = min(
              [keypoint.score for keypoint in person.keypoints])
          should_keep_image = min_landmark_score >= detection_threshold
          if not should_keep_image:
            self._messages.append('Skipped ' + image_path +
                                  '. No pose was confidentlly detected.')
            continue

          valid_image_count += 1

          # Draw the prediction result on top of the image for debugging later
          output_overlay = draw_prediction_on_image(
              image.numpy().astype(np.uint8), person,
              close_figure=True, keep_input_size=True)

          # Write detection result into an image file
          output_frame = cv2.cvtColor(output_overlay, cv2.COLOR_RGB2BGR)
          cv2.imwrite(os.path.join(images_out_folder, image_name), output_frame)

          # Get landmarks and scale it to the same size as the input image
          pose_landmarks = np.array(
              [[keypoint.coordinate.x, keypoint.coordinate.y, keypoint.score]
                for keypoint in person.keypoints],
              dtype=np.float32)

          # Write the landmark coordinates to its per-class CSV file
          coordinates = pose_landmarks.flatten().astype(str).tolist()
          csv_out_writer.writerow([image_name] + coordinates)

        if not valid_image_count:
          raise RuntimeError(
              'No valid images found for the "{}" class.'
              .format(pose_class_name))

    # Print the error message collected during preprocessing.
    print('\n'.join(self._messages))

    # Combine all per-class CSVs into a single output file
    all_landmarks_df = self._all_landmarks_as_dataframe()
    all_landmarks_df.to_csv(self._csvs_out_path, index=False)

  def class_names(self):
    """List of classes found in the training dataset."""
    return self._pose_class_names

  def _all_landmarks_as_dataframe(self):
    """Merge all per-class CSVs into a single dataframe."""
    total_df = None
    for class_index, class_name in enumerate(self._pose_class_names):
      csv_out_path = os.path.join(self._csvs_out_folder_per_class,
                                  class_name + '.csv')
      per_class_df = pd.read_csv(csv_out_path, header=None)

      # Add the labels
      per_class_df['class_no'] = [class_index]*len(per_class_df)
      per_class_df['class_name'] = [class_name]*len(per_class_df)

      # Append the folder name to the filename column (first column)
      per_class_df[per_class_df.columns[0]] = (os.path.join(class_name, '')
        + per_class_df[per_class_df.columns[0]].astype(str))

      if total_df is None:
        # For the first class, assign its data to the total dataframe
        total_df = per_class_df
      else:
        # Concatenate each class's data into the total dataframe
        total_df = pd.concat([total_df, per_class_df], axis=0)

    list_name = [[bodypart.name + '_x', bodypart.name + '_y',
                  bodypart.name + '_score'] for bodypart in BodyPart]
    header_name = []
    for columns_name in list_name:
      header_name += columns_name
    header_name = ['file_name'] + header_name
    header_map = {total_df.columns[i]: header_name[i]
                  for i in range(len(header_name))}

    total_df.rename(header_map, axis=1, inplace=True)

    return total_df

In [ ]:
is_skip_step_1 = False
use_custom_dataset = True

dataset_is_split = False

In [ ]:
import os
import random
import shutil

def split_into_train_test(images_origin, images_dest, test_split):
  """Splits a directory of sorted images into training and test sets.

  Args:
    images_origin: Path to the directory with your images. This directory
      must include subdirectories for each of your labeled classes. For example:
      yoga_poses/
      |__ downdog/
          |______ 00000128.jpg
          |______ 00000181.jpg
          |______ ...
      |__ goddess/
          |______ 00000243.jpg
          |______ 00000306.jpg
          |______ ...
      ...
    images_dest: Path to a directory where you want the split dataset to be
      saved. The results looks like this:
      split_yoga_poses/
      |__ train/
          |__ downdog/
              |______ 00000128.jpg
              |______ ...
      |__ test/
          |__ downdog/
              |______ 00000181.jpg
              |______ ...
    test_split: Fraction of data to reserve for test (float between 0 and 1).
  """
  _, dirs, _ = next(os.walk(images_origin))

  TRAIN_DIR = os.path.join(images_dest, 'train')
  TEST_DIR = os.path.join(images_dest, 'test')
  os.makedirs(TRAIN_DIR, exist_ok=True)
  os.makedirs(TEST_DIR, exist_ok=True)

  for dir in dirs:
    # Get all filenames for this dir, filtered by filetype
    filenames = os.listdir(os.path.join(images_origin, dir))
    filenames = [os.path.join(images_origin, dir, f) for f in filenames if (
        f.endswith('.png') or f.endswith('.jpg') or f.endswith('.jpeg') or f.endswith('.bmp'))]
    # Shuffle the files, deterministically
    filenames.sort()
    #random.seed(42)
    #random.shuffle(filenames)
    # Divide them into train/test dirs
    os.makedirs(os.path.join(TEST_DIR, dir), exist_ok=True)
    os.makedirs(os.path.join(TRAIN_DIR, dir), exist_ok=True)
    test_count = int(len(filenames) * test_split)
    for i, file in enumerate(filenames):
      if i < test_count:
        destination = os.path.join(TEST_DIR, dir, os.path.split(file)[1])
      else:
        destination = os.path.join(TRAIN_DIR, dir, os.path.split(file)[1])
      shutil.copyfile(file, destination)
    print(f'Moved {test_count} of {len(filenames)} from class "{dir}" into test.')
  print(f'Your split dataset is in "{images_dest}"')

In [ ]:
if use_custom_dataset:
  # ATTENTION:
  # You must edit these two lines to match your archive and images folder name:
  # !tar -xf YOUR_DATASET_ARCHIVE_NAME.tar
  #!unzip -q YOUR_DATASET_ARCHIVE_NAME.zip
  dataset_in = '/content/dataset'

  # You can leave the rest alone:
  if not os.path.isdir(dataset_in):
    raise Exception("dataset_in is not a valid directory")
  if dataset_is_split:
    IMAGES_ROOT = dataset_in
  else:
    dataset_out = 'splited_dataset'
    split_into_train_test(dataset_in, dataset_out, test_split=0.2)
    IMAGES_ROOT = dataset_out

Moved 17 of 88 from class "Incorrect" into test.
Moved 98 of 490 from class "correct" into test.
Your split dataset is in "splited_dataset"


In [ ]:
if not is_skip_step_1:
  images_in_train_folder = os.path.join(IMAGES_ROOT, '/content/splited_dataset/train')
  images_out_train_folder = 'poses_images_out_train'
  csvs_out_train_path = 'train_data.csv'

  preprocessor = MoveNetPreprocessor(
      images_in_folder=images_in_train_folder,
      images_out_folder=images_out_train_folder,
      csvs_out_path=csvs_out_train_path,
  )

  preprocessor.process(per_pose_class_limit=None)

Preprocessing Incorrect
100%|██████████| 65/65 [03:45<00:00,  3.46s/it]
Preprocessing correct
100%|██████████| 446/446 [26:16<00:00,  3.53s/it]

Skipped /content/splited_dataset/train/Incorrect/Xpull_up_2_frame_0000.jpg. No pose was confidentlly detected.
Skipped /content/splited_dataset/train/Incorrect/Xpull_up_2_frame_0012.jpg. No pose was confidentlly detected.
Skipped /content/splited_dataset/train/Incorrect/Xpull_up_2_frame_0120.jpg. No pose was confidentlly detected.
Skipped /content/splited_dataset/train/Incorrect/Xpull_up_2_frame_0126.jpg. No pose was confidentlly detected.
Skipped /content/splited_dataset/train/correct/pull up_1_frame_0090.jpg. No pose was confidentlly detected.
Skipped /content/splited_dataset/train/correct/pull up_1_frame_0155.jpg. No pose was confidentlly detected.
Skipped /content/splited_dataset/train/correct/pull up_1_frame_0160.jpg. No pose was confidentlly detected.
Skipped /content/splited_dataset/train/correct/pull up_1_frame_0390.jpg. No pose was confidentlly detected.
Skipped /content/splited_dataset/train/correct/pull up_1_frame_0495.jpg. No pose was confidentlly detected.
Skipped /content

In [ ]:
if not is_skip_step_1:
  images_in_test_folder = os.path.join(IMAGES_ROOT, '/content/splited_dataset/test')
  images_out_test_folder = 'poses_images_out_test'
  csvs_out_test_path = 'test_data.csv'

  preprocessor = MoveNetPreprocessor(
      images_in_folder=images_in_test_folder,
      images_out_folder=images_out_test_folder,
      csvs_out_path=csvs_out_test_path,
  )

  preprocessor.process(per_pose_class_limit=None)

Preprocessing Incorrect
  0%|          | 0/23 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 23/23 [01:20<00:00,  3.50s/it]
Preprocessing correct
100%|██████████| 44/44 [02:33<00:00,  3.49s/it]

Skipped /content/splited_dataset/test/Incorrect/Xpull_up_1_frame_0000.jpg. No pose was confidentlly detected.


In [ ]:
def load_pose_landmarks(csv_path):
  """Loads a CSV created by MoveNetPreprocessor.

  Returns:
    X: Detected landmark coordinates and scores of shape (N, 17 * 3)
    y: Ground truth labels of shape (N, label_count)
    classes: The list of all class names found in the dataset
    dataframe: The CSV loaded as a Pandas dataframe features (X) and ground
      truth labels (y) to use later to train a pose classification model.
  """

  # Load the CSV file
  dataframe = pd.read_csv(csv_path)
  df_to_process = dataframe.copy()

  # Drop the file_name columns as you don't need it during training.
  df_to_process.drop(columns=['file_name'], inplace=True)

  # Extract the list of class names
  classes = df_to_process.pop('class_name').unique()

  # Extract the labels
  y = df_to_process.pop('class_no')

  # Convert the input features and labels into the correct format for training.
  X = df_to_process.astype('float64')
  y = keras.utils.to_categorical(y)

  return X, y, classes, dataframe

In [ ]:
# Load the train data
X, y, class_names, _ = load_pose_landmarks(csvs_out_train_path)

# Split training data (X, y) into (X_train, y_train) and (X_val, y_val)
X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.15)

In [ ]:
# Load the test data
X_test, y_test, _, df_test = load_pose_landmarks(csvs_out_test_path)

In [ ]:
def get_center_point(landmarks, left_bodypart, right_bodypart):
  """Calculates the center point of the two given landmarks."""

  left = tf.gather(landmarks, left_bodypart.value, axis=1)
  right = tf.gather(landmarks, right_bodypart.value, axis=1)
  center = left * 0.5 + right * 0.5
  return center


def get_pose_size(landmarks, torso_size_multiplier=2.5):
  """Calculates pose size.

  It is the maximum of two values:
    * Torso size multiplied by `torso_size_multiplier`
    * Maximum distance from pose center to any pose landmark
  """
  # Hips center
  hips_center = get_center_point(landmarks, BodyPart.LEFT_HIP,
                                 BodyPart.RIGHT_HIP)

  # Shoulders center
  shoulders_center = get_center_point(landmarks, BodyPart.LEFT_SHOULDER,
                                      BodyPart.RIGHT_SHOULDER)

  # Torso size as the minimum body size
  torso_size = tf.linalg.norm(shoulders_center - hips_center)

  # Pose center
  pose_center_new = get_center_point(landmarks, BodyPart.LEFT_HIP,
                                     BodyPart.RIGHT_HIP)
  pose_center_new = tf.expand_dims(pose_center_new, axis=1)
  # Broadcast the pose center to the same size as the landmark vector to
  # perform substraction
  pose_center_new = tf.broadcast_to(pose_center_new,
                                    [tf.size(landmarks) // (17*2), 17, 2])

  # Dist to pose center
  d = tf.gather(landmarks - pose_center_new, 0, axis=0,
                name="dist_to_pose_center")
  # Max dist to pose center
  max_dist = tf.reduce_max(tf.linalg.norm(d, axis=0))

  # Normalize scale
  pose_size = tf.maximum(torso_size * torso_size_multiplier, max_dist)

  return pose_size


def normalize_pose_landmarks(landmarks):
  """Normalizes the landmarks translation by moving the pose center to (0,0) and
  scaling it to a constant pose size.
  """
  # Move landmarks so that the pose center becomes (0,0)
  pose_center = get_center_point(landmarks, BodyPart.LEFT_HIP,
                                 BodyPart.RIGHT_HIP)
  pose_center = tf.expand_dims(pose_center, axis=1)
  # Broadcast the pose center to the same size as the landmark vector to perform
  # substraction
  pose_center = tf.broadcast_to(pose_center,
                                [tf.size(landmarks) // (17*2), 17, 2])
  landmarks = landmarks - pose_center

  # Scale the landmarks to a constant pose size
  pose_size = get_pose_size(landmarks)
  landmarks /= pose_size

  return landmarks


def landmarks_to_embedding(landmarks_and_scores):
  """Converts the input landmarks into a pose embedding."""
  # Reshape the flat input into a matrix with shape=(17, 3)
  reshaped_inputs = keras.layers.Reshape((17, 3))(landmarks_and_scores)

  # Normalize landmarks 2D
  landmarks = normalize_pose_landmarks(reshaped_inputs[:, :, :2])

  # Flatten the normalized landmark coordinates into a vector
  embedding = keras.layers.Flatten()(landmarks)

  return embedding

In [ ]:
# Define the model
inputs = tf.keras.Input(shape=(51))
embedding = landmarks_to_embedding(inputs)

layer = keras.layers.Dense(128, activation=tf.nn.relu6)(embedding)
layer = keras.layers.Dropout(0.5)(layer)
layer = keras.layers.Dense(64, activation=tf.nn.relu6)(layer)
layer = keras.layers.Dropout(0.5)(layer)
outputs = keras.layers.Dense(len(class_names), activation="softmax")(layer)

model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 51)]                 0         []                            
                                                                                                  
 reshape (Reshape)           (None, 17, 3)                0         ['input_1[0][0]']             
                                                                                                  
 tf.__operators__.getitem (  (None, 17, 2)                0         ['reshape[0][0]']             
 SlicingOpLambda)                                                                                 
                                                                                                  
 tf.compat.v1.gather (TFOpL  (None, 2)                    0         ['tf.__operators__.getitem

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Add a checkpoint callback to store the checkpoint that has the highest
# validation accuracy.
checkpoint_path = "weights.best.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             mode='max')
earlystopping = keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                              patience=20)

# Start training
history = model.fit(X_train, y_train,
                    epochs=200,
                    batch_size=16,
                    validation_data=(X_val, y_val),
                    callbacks=[checkpoint, earlystopping])

Epoch 1/200
22/25 [=========================>....] - ETA: 0s - loss: 0.0380 - accuracy: 0.9972
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to weights.best.hdf5
25/25 [==============================] - 4s 46ms/step - loss: 0.0353 - accuracy: 0.9975 - val_loss: 0.0099 - val_accuracy: 1.0000
Epoch 2/200
24/25 [===========================>..] - ETA: 0s - loss: 0.0312 - accuracy: 0.9974

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_accuracy did not improve from 1.00000
25/25 [==============================] - 0s 8ms/step - loss: 0.0322 - accuracy: 0.9949 - val_loss: 0.0092 - val_accuracy: 1.0000
Epoch 3/200
24/25 [===========================>..] - ETA: 0s - loss: 0.0275 - accuracy: 1.0000
Epoch 3: val_accuracy did not improve from 1.00000
25/25 [==============================] - 0s 17ms/step - loss: 0.0268 - accuracy: 1.0000 - val_loss: 0.0062 - val_accuracy: 1.0000
Epoch 4/200
20/25 [=======================>......] - ETA: 0s - loss: 0.0191 - accuracy: 0.9969
Epoch 4: val_accuracy did not improve from 1.00000
25/25 [==============================] - 0s 13ms/step - loss: 0.0204 - accuracy: 0.9975 - val_loss: 0.0052 - val_accuracy: 1.0000
Epoch 5/200
23/25 [==========================>...] - ETA: 0s - loss: 0.0231 - accuracy: 0.9973
Epoch 5: val_accuracy did not improve from 1.00000
25/25 [==============================] - 0s 12ms/step - loss: 0.0222 - accuracy: 0.9975 - val_loss: 0.0048 - val_accuracy

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)

3/3 [==============================] - 0s 10ms/step - loss: 0.0934 - accuracy: 1.0000
Test Accuracy: 1.0


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

print('Model size: %dKB' % (len(tflite_model) / 1024))

with open('pose_classifierFinal.tflite', 'wb') as f:
  f.write(tflite_model)

Model size: 26KB


In [ ]:
# Convert items in class_names to strings
class_names_str = [str(item) for item in class_names]

# Write the string representation of class_names to the file
with open('pose_labels.txt', 'w') as f:
    f.write('\n'.join(class_names_str))


In [ ]:
!zip pose_classifier.zip pose_labels.txt pose_classifier.tflite


  adding: pose_labels.txt (deflated 24%)
  adding: pose_classifier.tflite (deflated 36%)


In [ ]:
try:
  from google.colab import files
  files.download('pose_classifier.zip')
except:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
images_in_train_folder = '/content/Run_frames'
images_out_train_folder = 'poses_images_out_run'
csvs_out_train_path = 'run_data.csv'

preprocessor = MoveNetPreprocessor(
      images_in_folder=images_in_train_folder,
      images_out_folder=images_out_train_folder,
      csvs_out_path=csvs_out_train_path,
  )

preprocessor.process(per_pose_class_limit=None)

Preprocessing Frames
100%|██████████| 30/30 [01:49<00:00,  3.65s/it]

In [ ]:
X_run, y_run, _, df_run = load_pose_landmarks("/content/run_data.csv")
inputs = tf.keras.Input(shape=(51))
embeddingg = landmarks_to_embedding(inputs)


In [ ]:
print(type(X_run))
np_Xarrayy = X_run.values
print(X_run)
print(np_Xarrayy)

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [ ]:
print((input_shape))

[ 1 51]
